Optimization

Preprocessing

In [18]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU, ReLU, Dropout

# Import and read in charity_data.csv
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

In [2]:
# Drop non-beneficial ID columns, 'EIN' and 'NAME'
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [3]:
# Determine the number of unique values in each column
for col in application_df.columns:
  print(f"{col}: {application_df[col].nunique()} unique values")

APPLICATION_TYPE: 17 unique values
AFFILIATION: 6 unique values
CLASSIFICATION: 71 unique values
USE_CASE: 5 unique values
ORGANIZATION: 4 unique values
STATUS: 2 unique values
INCOME_AMT: 9 unique values
SPECIAL_CONSIDERATIONS: 2 unique values
ASK_AMT: 8747 unique values
IS_SUCCESSFUL: 2 unique values


In [4]:
# Binning for APPLICATION_TYPE
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_cutoff = 500
application_types_to_replace = list(app_type_counts[app_type_counts < app_type_cutoff].index)

for app in application_types_to_replace:
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

In [5]:
# Binning for CLASSIFICATION
class_counts = application_df['CLASSIFICATION'].value_counts()
class_cutoff = 1800
classifications_to_replace = list(class_counts[class_counts < class_cutoff].index)

for cls in classifications_to_replace:
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

In [6]:
# Convert categorical data to numeric with 'pd.get_dummies'
application_df = pd.get_dummies(application_df)

In [7]:
# Split preprocessed data into features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# SPlit the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler isntance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Model Creation

In [19]:
#Design a neural network model with five hidden layers
nn_optimized = tf.keras.models.Sequential()

# First layer with Leaky ReLU activation
nn_optimized.add(tf.keras.layers.Dense(units=256, input_dim=len(X_train_scaled[0])))
nn_optimized.add(LeakyReLU(alpha=0.1))
nn_optimized.add(Dropout(0.3)) #Added dropout for regularization

# Second layer with ReLU activation
nn_optimized.add(tf.keras.layers.Dense(units=128))
nn_optimized.add(ReLU())
nn_optimized.add(Dropout(0.3)) #Added dropout for regularization

# Third layer with ELU activation
nn_optimized.add(tf.keras.layers.Dense(units=64))
nn_optimized.add(ELU())
nn_optimized.add(Dropout(0.3))  # Added dropout for regularization

# Fourth layer with ReLU activation
nn_optimized.add(tf.keras.layers.Dense(units=32))
nn_optimized.add(ReLU())
nn_optimized.add(Dropout(0.3))  # Added dropout for regularization

# Fifth layer with PReLU activation
nn_optimized.add(tf.keras.layers.Dense(units=16))
nn_optimized.add(PReLU())
nn_optimized.add(Dropout(0.3))  # Added dropout for regularization


# Output layer with sigmoid activation for binary classification
nn_optimized.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check structure of optimized model
nn_optimized.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 256)               11264     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_14 (Dense)            (None, 128)               32896     
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_15 (Dense)            (None, 64)               

In [20]:
# Compile the optimized model
nn_optimized.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [21]:
# Train the optimized model with 200 epochs
model_fit_optimized = nn_optimized.fit(X_train_scaled, y_train, epochs=200)


Epoch 1/200
804/804 [==============================] - 5s 5ms/step - loss: 0.6064 - accuracy: 0.6971
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5783 - accuracy: 0.7234
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5716 - accuracy: 0.7277
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5677 - accuracy: 0.7286
Epoch 5/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5689 - accuracy: 0.7288
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5657 - accuracy: 0.7302
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5631 - accuracy: 0.7308
Epoch 8/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5641 - accuracy: 0.7300
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5623 - accuracy: 0.7292
Epoch 10/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5618 - accura

In [22]:
# Evaluate the optimized model using the test data
model_loss_optimized, model_accuracy_optimized = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Model Loss: {model_loss_optimized}, Optimized Model Accuracy: {model_accuracy_optimized}")


268/268 - 1s - loss: 0.5587 - accuracy: 0.7257 - 559ms/epoch - 2ms/step
Optimized Model Loss: 0.5586734414100647, Optimized Model Accuracy: 0.7257142663002014


Further Optimization and another Model

In [23]:
# Second Optimization

# Design a neural network model with more hidden layers and different activation functions
nn_optimized_2 = tf.keras.models.Sequential()

# First layer with Leaky ReLU activation
nn_optimized_2.add(tf.keras.layers.Dense(units=256, input_dim=len(X_train_scaled[0])))
nn_optimized_2.add(LeakyReLU(alpha=0.1))
nn_optimized_2.add(Dropout(0.4))  # Adding dropout for regularization

# Second layer with ReLU activation
nn_optimized_2.add(tf.keras.layers.Dense(units=128))
nn_optimized_2.add(ReLU())
nn_optimized_2.add(Dropout(0.4))  # Adding dropout for regularization

# Third layer with ELU activation
nn_optimized_2.add(tf.keras.layers.Dense(units=64))
nn_optimized_2.add(ELU())
nn_optimized_2.add(Dropout(0.4))  # Adding dropout for regularization

# Fourth layer with SELU activation
nn_optimized_2.add(tf.keras.layers.Dense(units=32))
nn_optimized_2.add(tf.keras.layers.Activation('selu'))
nn_optimized_2.add(Dropout(0.4))  # Adding dropout for regularization

# Fifth layer with Swish activation
nn_optimized_2.add(tf.keras.layers.Dense(units=16))
nn_optimized_2.add(tf.keras.layers.Activation('swish'))
nn_optimized_2.add(Dropout(0.4))  # Adding dropout for regularization

# Sixth layer with PReLU activation
nn_optimized_2.add(tf.keras.layers.Dense(units=8))
nn_optimized_2.add(PReLU())
nn_optimized_2.add(Dropout(0.4))  # Adding dropout for regularization

# Seventh layer with ReLU activation
nn_optimized_2.add(tf.keras.layers.Dense(units=4))
nn_optimized_2.add(ReLU())
nn_optimized_2.add(Dropout(0.4))  # Adding dropout for regularization

# Output layer with sigmoid activation for binary classification
nn_optimized_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the optimized model
nn_optimized_2.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 256)               11264     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_20 (Dense)            (None, 128)               32896     
                                                                 
 re_lu_2 (ReLU)              (None, 128)               0         
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_21 (Dense)            (None, 64)               

In [24]:
# Compile the optimized model
nn_optimized_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [25]:
# Train the optimized model with 300 epochs
model_fit_optimized_2 = nn_optimized_2.fit(X_train_scaled, y_train, epochs=300)


Epoch 1/300
804/804 [==============================] - 6s 5ms/step - loss: 0.6867 - accuracy: 0.5639
Epoch 2/300
804/804 [==============================] - 3s 4ms/step - loss: 0.6485 - accuracy: 0.6297
Epoch 3/300
804/804 [==============================] - 3s 4ms/step - loss: 0.6287 - accuracy: 0.6677
Epoch 4/300
804/804 [==============================] - 4s 4ms/step - loss: 0.6160 - accuracy: 0.6820
Epoch 5/300
804/804 [==============================] - 4s 5ms/step - loss: 0.6180 - accuracy: 0.6828
Epoch 6/300
804/804 [==============================] - 4s 5ms/step - loss: 0.6096 - accuracy: 0.6937
Epoch 7/300
804/804 [==============================] - 4s 5ms/step - loss: 0.6054 - accuracy: 0.7001
Epoch 8/300
804/804 [==============================] - 4s 5ms/step - loss: 0.6079 - accuracy: 0.6954
Epoch 9/300
804/804 [==============================] - 3s 4ms/step - loss: 0.6019 - accuracy: 0.6982
Epoch 10/300
804/804 [==============================] - 3s 4ms/step - loss: 0.6013 - accura

In [26]:
# Evaluate the optimized model using the test data
model_loss_optimized_2, model_accuracy_optimized_2 = nn_optimized_2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Model Loss: {model_loss_optimized_2}, Optimized Model Accuracy: {model_accuracy_optimized_2}")


268/268 - 1s - loss: 0.5598 - accuracy: 0.7264 - 592ms/epoch - 2ms/step
Optimized Model Loss: 0.5597774982452393, Optimized Model Accuracy: 0.7264139652252197


In [29]:
# Third Optimization

# Design a neural network model with more hidden layers and different activation functions
nn_optimized_3 = tf.keras.models.Sequential()

# First layer with Leaky ReLU activation
nn_optimized_3.add(tf.keras.layers.Dense(units=512, input_dim=len(X_train_scaled[0])))
nn_optimized_3.add(LeakyReLU(alpha=0.2))
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Second layer with ReLU activation
nn_optimized_3.add(tf.keras.layers.Dense(units=256))
nn_optimized_3.add(ReLU())
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Third layer with SELU activation
nn_optimized_3.add(tf.keras.layers.Dense(units=128))
nn_optimized_3.add(tf.keras.layers.Activation('selu'))
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Fourth layer with Swish activation
nn_optimized_3.add(tf.keras.layers.Dense(units=64))
nn_optimized_3.add(tf.keras.layers.Activation('swish'))
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Fifth layer with PReLU activation
nn_optimized_3.add(tf.keras.layers.Dense(units=32))
nn_optimized_3.add(PReLU())
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Sixth layer with ELU activation
nn_optimized_3.add(tf.keras.layers.Dense(units=16))
nn_optimized_3.add(ELU())
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Seventh layer with Swish activation
nn_optimized_3.add(tf.keras.layers.Dense(units=8))
nn_optimized_3.add(tf.keras.layers.Activation('swish'))
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Eighth layer with Leaky ReLU activation
nn_optimized_3.add(tf.keras.layers.Dense(units=4))
nn_optimized_3.add(LeakyReLU(alpha=0.2))
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Ninth layer with ReLU activation
nn_optimized_3.add(tf.keras.layers.Dense(units=2))
nn_optimized_3.add(ReLU())
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Tenth layer with PReLU activation
nn_optimized_3.add(tf.keras.layers.Dense(units=1))
nn_optimized_3.add(PReLU())
nn_optimized_3.add(Dropout(0.3))  # Adding dropout for regularization

# Output layer with sigmoid activation for binary classification
nn_optimized_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the optimized model
nn_optimized_3.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 512)               22528     
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 512)               0         
                                                                 
 dropout_30 (Dropout)        (None, 512)               0         
                                                                 
 dense_48 (Dense)            (None, 256)               131328    
                                                                 
 re_lu_7 (ReLU)              (None, 256)               0         
                                                                 
 dropout_31 (Dropout)        (None, 256)               0         
                                                                 
 dense_49 (Dense)            (None, 128)              

In [30]:
# Compile the optimized model
nn_optimized_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [36]:
# Train the optimized model with 500 epochs
model_fit_optimized_3 = nn_optimized_3.fit(X_train_scaled, y_train, epochs=500)


Epoch 1/500
804/804 [==============================] - 9s 11ms/step - loss: 0.6293 - accuracy: 0.6498
Epoch 2/500
804/804 [==============================] - 7s 9ms/step - loss: 0.6311 - accuracy: 0.6489
Epoch 3/500
804/804 [==============================] - 9s 11ms/step - loss: 0.6309 - accuracy: 0.6484
Epoch 4/500
804/804 [==============================] - 7s 9ms/step - loss: 0.6333 - accuracy: 0.6488
Epoch 5/500
804/804 [==============================] - 8s 10ms/step - loss: 0.6330 - accuracy: 0.6454
Epoch 6/500
804/804 [==============================] - 8s 10ms/step - loss: 0.6306 - accuracy: 0.6497
Epoch 7/500
804/804 [==============================] - 7s 9ms/step - loss: 0.6330 - accuracy: 0.6473
Epoch 8/500
804/804 [==============================] - 8s 10ms/step - loss: 0.6281 - accuracy: 0.6546
Epoch 9/500
804/804 [==============================] - 7s 9ms/step - loss: 0.6302 - accuracy: 0.6466
Epoch 10/500
804/804 [==============================] - 8s 10ms/step - loss: 0.6352 - 

In [37]:
# Evaluate the optimized model using the test data
model_loss_optimized_3, model_accuracy_optimized_3 = nn_optimized_3.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Model Loss: {model_loss_optimized_3}, Optimized Model Accuracy: {model_accuracy_optimized_3}")


268/268 - 1s - loss: 0.6084 - accuracy: 0.7245 - 641ms/epoch - 2ms/step
Optimized Model Loss: 0.6083703637123108, Optimized Model Accuracy: 0.7245481014251709


In [38]:
# Export the further optimized model to HDF5 file
nn_optimized_3.save("AlphabetSoupCharity_Optimization.h5")
